In [6]:
import os
from sqlalchemy import create_engine, text

results = None
with create_engine(os.environ["REMOTE_POSTGRES_URL"]).connect() as conn:
    result = conn.execute(text("select * from users where id=15972102"))
    results = result.all()
results

[(15972102, 'bfef', '👣', 'Efrem', 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/15972102/9131294/7/medium.jpg', '{"access_token": "c99888dc477be6a726bc7f5dcc78d84a2844d7dd", "refresh_token": "05867993a2d0c5b60c51653636a9c295348551f3", "expires_at": 1643749227}', None, 'Oakland', 'California', 'United States', None, datetime.datetime(2022, 1, 28, 22, 18, 9, 696673), 5194, False, None)]

In [7]:
import numpy as np
USER_DTYPE = np.dtype([
    ("uid", np.uint32), 
    ("username", object), 
    ("firstname", object), 
    ("lastname", str), 
    ("profile", str), 
    ("access_token", str), 
    ("measurement_preference", str),
    ("city", str),
    ("state", str),
    ("country", str),
    ("email", str),
    ("dt_last_active", str),
    ("dt_indexed", str),
    ("app_activity_count", str),
    ("share_profile", bool),
    ("extra", str)
])
results2 = np.zeros(len(results), dtype=USER_DTYPE)

In [8]:
import motor.motor_asyncio
mongo_url = os.environ.get("REMOTE_MONGODB_URL")
client = motor.motor_asyncio.AsyncIOMotorClient(mongo_url)

In [9]:
db = client.get_default_database()
db

AsyncIOMotorDatabase(Database(MongoClient(host=['cluster0-shard-00-02-wlylc.mongodb.net:27017', 'cluster0-shard-00-00-wlylc.mongodb.net:27017', 'cluster0-shard-00-01-wlylc.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=False, retrywrites=True, w='majority', authsource='admin', replicaset='Cluster0-shard-0', ssl=True, driver=DriverInfo(name='Motor', version='2.5.1', platform='asyncio')), 'heatflask'))

In [10]:
import asyncio
from pprint import pprint
names = await db.list_collection_names()
pprint(names)

cursor = db["index"].find()
index = await cursor.to_list(length=100)

['activities', 'payments', 'history', 'index', 'updates']


In [11]:
index

[{'_id': 1446510156,
  'ts_UTC': '2018-03-10T22:11:57Z',
  'total_distance': 45974.3,
  'start_latlng': [35.067757, 136.680551],
  'user_id': 18556576,
  'bounds': {'SW': [35.06775, 136.65741], 'NE': [35.24139, 136.69591]},
  'ts': datetime.datetime(2022, 1, 30, 11, 21, 5, 643000),
  'ts_local': datetime.datetime(2018, 3, 11, 7, 11, 57),
  'average_speed': 7.205,
  'elapsed_time': 6527,
  'type': 'Ride',
  'name': '全員集合 (背割)'},
 {'_id': 2053955858,
  'ts_UTC': '2019-01-03T22:49:07Z',
  'total_distance': 32381.4,
  'start_latlng': [35.071025, 136.691339],
  'user_id': 18556576,
  'bounds': {'SW': [35.06805, 136.62718], 'NE': [35.17931, 136.69607]},
  'ts': datetime.datetime(2022, 1, 30, 11, 21, 5, 643000),
  'ts_local': datetime.datetime(2019, 1, 4, 7, 49, 7),
  'average_speed': 7.9,
  'elapsed_time': 5668,
  'type': 'Ride',
  'name': '初モーニング (美濃石津→多度)'},
 {'_id': 2880879934,
  'ts_UTC': '2019-11-21T11:41:21Z',
  'total_distance': 37989.9,
  'start_latlng': [35.071122, 136.691232],
  'u

In [12]:
activities = await db["activities"].find().limit(10).to_list(length=10)

In [13]:
import msgpack as mp
import numpy as np 
import polyline

def rll_decode(rll_encoded, first_value=0):
    running_sum = first_value
    out_list = [first_value]

    for el in rll_encoded:
        if isinstance(el, list) and len(el) == 2:
            val, num_repeats = el
            for i in range(num_repeats):
#                 running_sum += val
#                 out_list.append(running_sum)
                out_list.append(val)
        else:
            out_list.append(el)
#             running_sum += el
#             out_list.append(running_sum)

    return out_list

LATLNG_DTYPE = np.dtype([("dt", np.uint8), ("lat", np.float32), ("lng", np.float32)])
A = mp.unpackb(activities[0]["mpk"])

lat, lng = zip(*polyline.decode(A["polyline"]))
dt = rll_decode(A["time"])
tlatlng = np.array(list(zip(dt, lat, lng)), LATLNG_DTYPE)
                

In [39]:
tlatlng, tlatlng.nbytes

(array([( 0, 35.07114, 136.69131), ( 1, 35.07112, 136.69133),
        ( 4, 35.07099, 136.69131), ( 5, 35.0708 , 136.69127),
        ( 7, 35.07052, 136.69122), ( 6, 35.07023, 136.69118),
        ( 7, 35.06984, 136.69112), (15, 35.06932, 136.69102),
        ( 8, 35.06885, 136.69093), ( 8, 35.0684 , 136.69086),
        ( 1, 35.06835, 136.69086), ( 8, 35.0679 , 136.69077),
        ( 9, 35.06743, 136.69067), ( 7, 35.06712, 136.69063),
        ( 6, 35.06688, 136.6906 ), ( 2, 35.0668 , 136.69061),
        ( 5, 35.06656, 136.69057), ( 2, 35.06644, 136.69054),
        ( 2, 35.06632, 136.69052), ( 1, 35.06625, 136.6905 ),
        ( 7, 35.0658 , 136.6904 ), ( 8, 35.06535, 136.69029),
        ( 7, 35.06503, 136.69022), ( 2, 35.06495, 136.69014),
        ( 1, 35.06495, 136.6901 ), ( 1, 35.06496, 136.69003),
        ( 2, 35.06501, 136.68993), ( 8, 35.06523, 136.68945),
        ( 6, 35.06532, 136.68909), ( 9, 35.06545, 136.68849),
        ( 7, 35.06555, 136.68797), ( 7, 35.06563, 136.68742),
        

In [40]:
activities[0]["mpk"], len(activities[0]["mpk"])

(b'\x83\xa8polyline\xda\x011sy`uEwohaYBAXBd@Fv@Fx@HlAJfBR|APxALH@xAN|AP|@Hn@DNAn@HVBVBL@xATxAT~@NNL?HAJITk@zAQhAYvBSdBOlBIr@Gp@[fCGh@Ih@Eh@Et@Gl@Mh@?FNJ|@Zf@JPDFDPJFBHJ?F?JG`@Ip@DHZPH@n@HTFzAV~AXfBTfARjARrARb@Hx@LlARH@v@JbBVHBpARtB\\J@rARZFDDBZOp@IZMh@g@jBQv@QbAEZGlABfAAn@?f@EdBAlA?\\BFPTFJDBB@CJ?JEJOFAD?F@`@DXBb@BTFj@Ht@@TCPEDE@K?I@\xa1nt\xa4time\xdc\x00o\x01\x04\x05\x07\x06\x07\x0f\x08\x08\x01\x08\t\x07\x06\x02\x05\x02\x02\x01\x07\x08\x07\x02\x01\x01\x02\x08\x06\t\x07\x07\x03\x03\t\x92\x03\x03\x04\x04\x05\x01\x02\x06\x04\x02\x01\x02\x01\x02\x01\x01\x03\x04\x01\x03\x01\x04\x02\x08\x08\t\x06\x06\x07\x03\x05\x07\x01\x05\t\x01\x07\n\x01\x08\x03\x01\x03\x04\x02\x03\x08\x04\x05\x02\x06\x06\x04\x03\x07\x06\x03\x01\x04\x02\x02\x03\x05\x92\x03\x03\x01\x01\x03\x02\x03\x02\x04\x05\x02\x02\x01\x01\x02\x03',
 444)

In [19]:
import aioredis
redis_url = os.environ.get("REMOTE_REDIS_URL")
redis = aioredis.from_url(redis_url)

In [23]:
key = await redis.randomkey()
val = await redis.get(key)

In [26]:
key, mp.unpackb(val)

(b'A:509676982',
 {'polyline': 'ghztEwg}bYJDVJJFLFLFNHLHNDPDNDNFPDNDPDNDNDNHLHNJLLHNJNFPFRFRDRLf@Hh@DRDRHh@Jf@Hd@Fb@H\\F^F^FXBH@FF`@BLFXFZFX@J@F@R@J?FHZBNNb@TZl@P`@NDBd@LDBTFVFh@H\\JL@x@D~AMbB]`BYpBe@~A]hAWz@UPGXGB?TGFCt@OxA[hAW~A[bBSz@KrB[REnBYPClBUb@EvBU^En@Kb@INCDAHAt@GnBUNCPAtBW~@MjBUl@IjBSz@KjBUlAO\\EjBS\\GjBUn@IzAQn@KrBg@bAYDCXKB?HETOTSHIPYx@eAHIX_@LSBCV[FI^e@z@eA`AkAf@g@PSX[bAgAhAoAhAoAp@{@JMfAwAX]pAgBFG?C??BAFEDIPSt@}@dAiAZ_@h@k@`AmAJMj@o@f@k@f@i@FIl@m@NQFQDKNNBBLRZ`@RVbApArA~Ab@h@~@xAN\\X`A^nAJ^Rl@Pl@X~@DNRn@d@|AJ\\L\\f@bBFPTpAHtB?V?lA@nADp@Hl@Fh@Hh@Ff@Fd@Jx@Lx@NdALxA@RBp@BXDd@?jBGjBO`BCRER]rBMf@g@xBKf@S|AEl@?D?DALCb@I~AA`@Ev@EpAE`AM|BCtA?tCBv@BdA?P@r@Bp@?P@r@@r@Bt@?P@`@@t@Bt@?N@r@BbA@r@FlC@PD~A@Z?F?T?l@@T@p@@vA@~@@vABlB@vABdABjCBbC@`ADhC@d@?N@d@?D@h@@`A@nABvC?j@@|C?T@dA@dA@jC?pB@|A?h@AL?B?D?D?\\Bz@@pA@lC@V@p@?vCBlCBrB?V?F?L?J?r@BvC?T?`B@rBDhC?fABlC@~A@`@?B?H?`@?hA@dCBlBErC?TI`BGd@QfAIb@Mr@c@|BIb@CNSfASjAe@hCETe@jCERKt@Mn@E\\AFCLCJMj@UjAc@fCEV[dBMt@Mv@YnBO|@Mp@Kb@Gd@Mv@Kp@WxBG